In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [2]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import (
    RichProgressBar,
    ModelCheckpoint,
    EarlyStopping,
)

from models.models import LitXLMRobertaModel
from models.data import ArabicAbstractsDataModule, AraSumDataModule

In [3]:
data_module = ArabicAbstractsDataModule(balance_ai_with_human=True)

In [4]:
data_module.setup()
len(data_module.train_dataset),len(data_module.val_dataset),len(data_module.test_dataset)

(4194, 899, 899)

check LLM data

In [7]:
set([i for i in data_module.labels if i > 0])

{1}

In [5]:
model = LitXLMRobertaModel()

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# # Add EarlyStopping and ModelCheckpoint callbacks
early_stopping_callback = EarlyStopping(
    monitor="val_loss",  # Monitor validation loss
    min_delta=0.0,  # Minimum change to qualify as improvement
    patience=3,  # Stop training after this epochs without improvement
    verbose=True,  # Print information at each validation step
    mode="min",  # Mode to minimize the monitored metric
)

In [7]:
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",  # Monitor validation loss
    dirpath="trained_detectors/Arabic/AraSumDetectorContinualTrainingOnArabicAbstracts/checkpoints",  # Directory to save checkpoints
    filename="best-checkpoint",  # Filename for the best checkpoint
    save_top_k=1,  # Save only the best checkpoint
    mode="min",  # Mode to minimize the monitored metric
)

In [8]:
# # Initialize a trainer with callbacks
trainer = pl.Trainer(
    devices=1,
    max_epochs=100,
    accelerator="auto",
    val_check_interval=0.25,
    check_val_every_n_epoch=1,
    callbacks=[early_stopping_callback, checkpoint_callback],
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [9]:
model = model.__class__.load_from_checkpoint(
    "trained_detectors/Arabic/AraSum/FromAllModelsAIDetector/checkpoints/best-checkpoint.ckpt",
    confusion_matrix_on_testing=True
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
trainer.test(model,datamodule=data_module)

You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=254` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5750834345817566     │
│          test_f1          │    0.6872767210006714     │
│         test_loss         │    1.9591894149780273     │
│      test_precision       │    0.5296081900596619     │
│        test_recall        │     0.996662974357605     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.9591894149780273,
  'test_acc': 0.5750834345817566,
  'test_precision': 0.5296081900596619,
  'test_recall': 0.996662974357605,
  'test_f1': 0.6872767210006714}]

In [11]:
# # Train the model
trainer.fit(model, datamodule=data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name            | Type                                | Params | Mode 
---------------------------------------------------------------------------------
0  | val_accuracy    | BinaryAccuracy                      | 0      | train
1  | test_accuracy   | BinaryAccuracy                      | 0      | train
2  | train_accuracy  | BinaryAccuracy                      | 0      | train
3  | xlm_roberta     | XLMRobertaForSequenceClassification | 278 M  | eval 
4  | fc              | Linear                              | 3      | train
5  | activation      | Sigmoid                             | 0      | train
6  | train_precision | BinaryPrecision                     | 0      | train
7  | val_precision   | BinaryPrecision                     | 0      | train
8  | test_precision  | BinaryPrecision                     | 0      | train
9  | train_recall    | BinaryRecall                        | 0      | train
10 | val_recall      | BinaryRecall    

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=254` in the `DataLoader` to improve performance.
/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=254` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Metric val_loss improved by 0.002 >= min_delta = 0.0. New best score: 0.008


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.008. Signaling Trainer to stop.


In [12]:
model = model.__class__.load_from_checkpoint("trained_detectors/Arabic/AraSumDetectorContinualTrainingOnArabicAbstracts/checkpoints/best-checkpoint.ckpt")

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
trainer.test(model,datamodule=data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.996662974357605     │
│          test_f1          │    0.9937248229980469     │
│         test_loss         │    0.01238589733839035    │
│      test_precision       │    0.9947895407676697     │
│        test_recall        │    0.9928203225135803     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.01238589733839035,
  'test_acc': 0.996662974357605,
  'test_precision': 0.9947895407676697,
  'test_recall': 0.9928203225135803,
  'test_f1': 0.9937248229980469}]

### Test again on AraSum

In [16]:
arasum_data_module = AraSumDataModule()

In [17]:
trainer.test(model,datamodule=arasum_data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=254` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9599999785423279     │
│          test_f1          │    0.9558005332946777     │
│         test_loss         │    0.13325004279613495    │
│      test_precision       │            1.0            │
│        test_recall        │    0.9176021218299866     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.13325004279613495,
  'test_acc': 0.9599999785423279,
  'test_precision': 1.0,
  'test_recall': 0.9176021218299866,
  'test_f1': 0.9558005332946777}]